In [1]:
from general_imports import *

In [2]:
df = pd.read_csv("data/products_cellphones_6_3.csv")
df.head(n=5)

,ID,Title,Link
0,0,iPhone 15 Pro Max 256GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-15-pro-max.html
1,1,iPhone 13 128GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-13.html
2,2,iPhone 14 Pro Max 128GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-14-pro-max.html
3,3,iPhone 15 128GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-15.html
4,4,iPhone 14 128GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-14.html


Perform crawling for each link and extract the data from the page.

In [3]:
def init_driver(url):
    service = Service(executable_path="chromedriver.exe")

    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = webdriver.Chrome(
        service=service,
        options=options)
    
    driver.get(url)
    driver.maximize_window()

    return driver

def move_to_element(driver, element):
    action = ActionChains(driver)
    element = driver.find_element(By.CLASS_NAME, element)
    action.move_to_element(element)
    
    action.perform()

def save_link_to_product(driver, link, name):
    # brand name
    brand_name = name.split(" ")[0]

    # for oppo reno 11 f only (special page)
    if link == "https://cellphones.com.vn/dien-thoai-oppo-reno-11-f.html":
        move_to_element(driver, 'cps-container')
        time.sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # original price 
        span_tag = soup.find('span', {'class': 'text-price'})
        original_price = span_tag.strong.get_text().replace("đ", "").replace(".", "").strip()

        # product info
        ul_element = soup.find('ul', {'class': 'technical-content'})
        li_elements = ul_element.find_all('li')

        data_info = {}
        
        for li in li_elements: 
            p_tag = li.find('p')
            div_tag = li.find('div')

            key = p_tag.get_text()
            value = div_tag.get_text()

            data_info[key] = value
        
        result = [name, brand_name, original_price, None, None, None, data_info, link]
        print(result)
        driver.close()
        return result

    # scroll down to get the needed information
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@class="box-more-promotion my-3"]'))
    )
    driver.execute_script("arguments[0].scrollIntoView();", element)

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@class="block-same-product"]'))
    )
    driver.execute_script("arguments[0].scrollIntoView();", element)

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@class="technical-content"]'))
    )
    driver.execute_script("arguments[0].scrollIntoView();", element)

    # Get the page source
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # original price (tpt---sale-price) // product__price--show
    try:
        original_price = soup.find_all('p', {"class": 'tpt---sale-price'})
        if len(original_price) > 0:
            original_price = soup.find_all('p', {"class": 'tpt---sale-price'})[1].get_text()\
                .replace("đ", "").replace(".", "").strip()
        else:
            original_price = soup.find('p', {"class": 'product__price--show'}).get_text()\
                .replace("đ", "").replace(".", "").strip()
    except:
        original_price = None

    # discount price (tpt---price) // product__price--through
    try:
        discount_price = soup.find_all('p', {"class": 'tpt---price'})
        if len(discount_price) > 0:
            discount_price = soup.find_all('p', {"class": 'tpt---price'})[0].get_text()\
                .replace("đ", "").replace(".", "").strip()
        else:
            discount_price = soup.find('p', {"class": 'product__price--through'}).get_text()\
                .replace("đ", "").replace(".", "").strip()
    except:
        discount_price = None
        
    # star_rating
    try:
        star_rating = soup.find('p', {"class": 'title is-4 m-0 p-0'}).get_text()
    except AttributeError:
        star_rating = None

    # rewview_count
    try:
        review_count = soup.find('p', {"class": 'boxReview-score__count'})
        review_count = review_count.find('strong').get_text()
    except AttributeError:
        review_count = None

    # product info
    ul_element = soup.find('ul', {'class': 'technical-content'})
    li_elements = ul_element.find_all('li')

    data_info = {}

    for li in li_elements: 
        p_tag = li.find('p')
        div_tag = li.find('div')

        key = p_tag.get_text(strip=True)
        value = div_tag.get_text(strip=True)

        data_info[key] = value

    result = [name, brand_name, discount_price, original_price,  
              star_rating, review_count, data_info, link]
    
    print(result)
    driver.close()
    return result

For testing 

In [4]:
# link = "https://cellphones.com.vn/samsung-galaxy-a54-8gb-256gb.html"
# link1 = "https://cellphones.com.vn/iphone-15-plus.html"
# link2 = "https://cellphones.com.vn/vivo-v25-pro.html"
# link3 = "https://cellphones.com.vn/sharp-rouvo-v.html"

# driver = init_driver(link2)
# cuu = save_link_to_product(driver, link2, "Oppo Reno 11F")

In [5]:
product_info_header = \
    ["product_name", "brand_name", "original_price", "discount_price", 
    "rating", "review_count", "specs_info", "link"]

df_products = pd.DataFrame(columns=product_info_header)
df_products.to_csv("data/product_info.csv", header=True, index=False)

count = 0
data = []

for index, row in df.iterrows():
    link, name = row["Link"], row["Title"]

    driver = init_driver(link)
    new_product = save_link_to_product(driver, link, name)

    data.append(new_product)

    print(f"Done row {count}/{len(df)}")

    count += 1
    if count % 2 == 0:
        df_new_products = pd.DataFrame(data, columns=product_info_header)
        # for row in data:
            # df_products.loc[len(df_products)] = row
        df_new_products = pd.concat([df_products, df_new_products], ignore_index=True)
        df_new_products.to_csv("data/product_info.csv", mode='a', encoding="utf-8", header=False, index=False)
      
        # print(len(data))
        data = []
        # print(len(data))

['iPhone 15 Pro Max 256GB | Chính hãng VN/A', 'iPhone', '34990000', '31390000', '5.0/5', '13', {'Kích thước màn hình': '6.7 inches', 'Công nghệ màn hình': 'Super Retina XDR OLED', 'Camera sau': 'Camera chính: 48MP, 24 mm, ƒ/1.78Camera góc siêu rộng: 12 MP, 13 mm, ƒ/2.2Camera Tele: 12 MP', 'Camera trước': '12MP, ƒ/1.9', 'Chipset': 'Apple A17 Pro 6 nhân', 'Dung lượng RAM': '8 GB', 'Bộ nhớ trong': '256 GB', 'Pin': '4422 mAh', 'Thẻ SIM': '2 SIM (nano‑SIM và eSIM)', 'Hệ điều hành': 'iOS 17', 'Độ phân giải màn hình': '2796 x 1290-pixel', 'Tính năng màn hình': 'Tốc độ làm mới 120Hz460 ppi2000 nitsHDRTrue ToneDải màu rộng (P3)Haptic TouchTỷ lệ tương phản 2.000.000:1'}, 'https://cellphones.com.vn/iphone-15-pro-max.html']
Done row 0/1056
['iPhone 13 128GB | Chính hãng VN/A', 'iPhone', '18990000', '14490000', '5.0/5', '151', {'Kích thước màn hình': '6.1 inches', 'Công nghệ màn hình': 'Super Retina XDR OLED', 'Camera sau': 'Camera góc rộng: 12MP, f/1.6Camera góc siêu rộng: 12MP, ƒ/2.4', 'Camera tr

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7FA38AD22+56930]
	(No symbol) [0x00007FF7FA2FF622]
	(No symbol) [0x00007FF7FA1B42E5]
	(No symbol) [0x00007FF7FA1F98ED]
	(No symbol) [0x00007FF7FA1F9A2C]
	(No symbol) [0x00007FF7FA23A967]
	(No symbol) [0x00007FF7FA21BCDF]
	(No symbol) [0x00007FF7FA2381E2]
	(No symbol) [0x00007FF7FA21BA43]
	(No symbol) [0x00007FF7FA1ED438]
	(No symbol) [0x00007FF7FA1EE4D1]
	GetHandleVerifier [0x00007FF7FA706AAD+3709933]
	GetHandleVerifier [0x00007FF7FA75FFED+4075821]
	GetHandleVerifier [0x00007FF7FA75817F+4043455]
	GetHandleVerifier [0x00007FF7FA429756+706710]
	(No symbol) [0x00007FF7FA30B8FF]
	(No symbol) [0x00007FF7FA306AE4]
	(No symbol) [0x00007FF7FA306C3C]
	(No symbol) [0x00007FF7FA2F68F4]
	BaseThreadInitThunk [0x00007FFD7377257D+29]
	RtlUserThreadStart [0x00007FFD741EAA58+40]
